# Slack

本笔记本展示如何使用Slack聊天加载器。这个类可以帮助将导出的Slack对话映射到LangChain聊天消息。

该过程包含三个步骤：
1. 按照[这里的说明](https://slack.com/help/articles/1500001548241-Request-to-export-all-conversations)导出所需的对话线程。
2. 创建`SlackChatLoader`，并将文件路径指向JSON文件或包含JSON文件的目录。
3. 调用`loader.load()`（或`loader.lazy_load()`）执行转换。可以选择使用`merge_chat_runs`来合并来自同一发送者的连续消息，和/或使用`map_ai_messages`将指定发送者的消息转换为"AIMessage"类。

## 1. 创建消息导出

目前（2023/08/23）此加载器最适合支持从Slack导出的直接消息对话格式的zip目录文件。请遵循Slack提供的最新说明进行操作。

我们在LangChain仓库中有一个示例。

In [1]:
import requests

permalink = "https://raw.githubusercontent.com/langchain-ai/langchain/342087bdfa3ac31d622385d0f2d09cf5e06c8db3/libs/langchain/tests/integration_tests/examples/slack_export.zip"
response = requests.get(permalink)
with open("slack_dump.zip", "wb") as f:
    f.write(response.content)

## 2. 创建聊天加载器

为加载器提供zip目录的文件路径。您可以选择指定映射到AI消息的用户ID，以及配置是否合并消息序列。

In [2]:
from langchain_community.chat_loaders.slack import SlackChatLoader

In [3]:
loader = SlackChatLoader(
    path="slack_dump.zip",
)

## 3. 加载消息

`load()`（或`lazy_load`）方法返回"ChatSessions"列表，目前每个加载的对话仅包含一个消息列表。

In [4]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "U0500003428" to AI messages
messages: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="U0500003428")
)

### 后续步骤

然后，您可以根据需要使用这些消息，例如微调模型、少量示例选择或直接为下一条消息进行预测。

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[1]["messages"]):
    print(chunk.content, end="", flush=True)

Hi, 

I hope you're doing well. I wanted to reach out and ask if you'd be available to meet up for coffee sometime next week. I'd love to catch up and hear about what's been going on in your life. Let me know if you're interested and we can find a time that works for both of us. 

Looking forward to hearing from you!

Best, [Your Name]